In [6]:
import os
import sys
import random
import math
import skimage.io
import re
import time
import numpy as np
from samples.coco import coco
from samples.shapes import shapes
import tensorflow as tf

import matplotlib.pyplot as plt
from config import Config
import matplotlib.patches as patches

%matplotlib inline
# import matplotlib
# matplotlib.use('TkAgg')
# import matplotlib.pyplot as plt


ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)


from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


MODEL_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_WEIGHT_PATH = os.path.join(ROOT_DIR, "logs/mask_rcnn_shapes_0040.h5")
IMAGE_DIR = os.path.join(ROOT_DIR, "ce_shi_ji")


#config = coco.CocoConfig()


class somethingConfig(Config):
    NAME = "something"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 1


class InferenceConfig(somethingConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceConfig()
config.display()


TEST_MODE = "inference"


def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax



dataset = coco.CocoDataset()
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))


model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(MODEL_WEIGHT_PATH, by_name=True)
class_names = ['BG', 'fingerprint1']
file_names = next(os.walk(IMAGE_DIR))[2]


for x in range(len(file_names)):
    image_t = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[x]))
    results = model.detect([image_t], verbose=1)
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image_t, r['rois'], r['masks'], r['class_ids'],
                                class_names, r['scores'], ax=ax, title="Predictions")


image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask = \
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,
                                       dataset.image_reference(image_id)))


log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)



ModuleNotFoundError: No module named 'samples'